In [ ]:
import numpy as np
import cv2 as cv2
import tensorflow as tf

from random import shuffle

from tqdm import tqdm
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

Pre-Processing IMG

In [ ]:
#travel to dir of image
width = 224
num_class = 2

trainpath = 'ripenessTest/labelData/'
#testpath = 'Mold/TrainMold/'


trainImg = [trainpath + f for f in os.listdir(trainpath)]
#testImg = [testpath + f for f in os.listdir(testpath)]

shuffle(trainImg)

print(trainImg)

In [56]:
#to label to answer of each img (function)
def getlabel(imgName):
   l = imgName.split('/')[2]
   l = l.split("(")[0]
   #
   if(l == 'Ok'):
     return(np.array([1,0]))
   elif(l == 'Bad'):
     return(np.array([0,1]))


In [57]:
#read img file and append in array (function)
def img2data(path):
  trainData = []
  for item in tqdm((path)):
      img = cv2.imread(item)
      img = cv2.resize(img, (width,width) )

      trainData.append([np.array(img), getlabel(item)])
      #plt.imshow(img)
      #plt.show()
  return trainData


In [58]:
#read img file and append in array
trainSet = img2data(trainImg)

100%|██████████| 1754/1754 [00:05<00:00, 327.37it/s]


In [59]:
#Split Test and Train dataset
train = trainSet[:-500]
test = trainSet[-500:]

print(len(train))
print(len(test))

xtrain = np.array([i[0] for i in train]).reshape(-1, width, width, 3)/255
ytrain = np.array([i[1] for i in train])

xtest = np.array([i[0] for i in test]).reshape(-1, width, width, 3)/255
ytest = np.array([i[1] for i in test])

1254
500


*End PreProcess IMG*

*MobileNetV2*

In [60]:
baseMobileNetV2Model = tf.keras.applications.MobileNetV2(input_shape = (224,224,3), include_top = False, weights="imagenet")
baseMobileNetV2Model.trainable = False

avgPoolingLayer =  tf.keras.layers.GlobalAveragePooling2D()(baseMobileNetV2Model.output)
predLayer = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer)

MobileNetV2model = tf.keras.models.Model(inputs = baseMobileNetV2Model.input, outputs = predLayer)

In [61]:
MobileNetV2model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
MobileNetV2model.fit(xtrain, ytrain, epochs=2 , validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
13/13 [==============================] - 40s 3s/step - loss: 0.5382 - accuracy: 0.7352 - val_loss: 0.4282 - val_accuracy: 0.8600
Epoch 2/2
13/13 [==============================] - 31s 2s/step - loss: 0.4008 - accuracy: 0.9075 - val_loss: 0.3357 - val_accuracy: 0.9360


In [62]:
pred = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(MobileNetV2model.predict(rimg))
   if result == 0:
      pred.append([1,0])
   if result == 1:
      pred.append([0,1])


print(classification_report(pred, ytest))



1/1 [==============================] - 0s 94ms/step
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       253
           1       0.93      0.94      0.94       247

   micro avg       0.94      0.94      0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500
 samples avg       0.94      0.94      0.94       500



*Finished*

DenseNet

In [63]:
baseDenseModel = tf.keras.applications.DenseNet121(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseDenseModel.trainable = False

In [64]:
avgPoolingLayerDen =  tf.keras.layers.GlobalAveragePooling2D()(baseDenseModel.output)
predLayerDen = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerDen)

DenseNetmodel = tf.keras.models.Model(inputs = baseDenseModel.input, outputs = predLayerDen)

In [65]:
DenseNetmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
DenseNetmodel.fit(xtrain, ytrain, epochs=5, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/5
13/13 [==============================] - 160s 12s/step - loss: 0.8025 - accuracy: 0.5175 - val_loss: 0.6473 - val_accuracy: 0.5780
Epoch 2/5
13/13 [==============================] - 143s 11s/step - loss: 0.6172 - accuracy: 0.6372 - val_loss: 0.5540 - val_accuracy: 0.7360
Epoch 3/5
13/13 [==============================] - 145s 11s/step - loss: 0.5372 - accuracy: 0.7552 - val_loss: 0.4946 - val_accuracy: 0.7980
Epoch 4/5
13/13 [==============================] - 145s 11s/step - loss: 0.4801 - accuracy: 0.8198 - val_loss: 0.4435 - val_accuracy: 0.8580
Epoch 5/5
13/13 [==============================] - 144s 11s/step - loss: 0.4312 - accuracy: 0.8708 - val_loss: 0.3986 - val_accuracy: 0.8980


In [66]:
predDen = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(DenseNetmodel.predict(rimg))
   if result == 0:
      predDen.append([1,0])
   if result == 1:
      predDen.append([0,1])

print(classification_report(predDen, ytest))

1/1 [==============================] - 0s 381ms/step
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       258
           1       0.88      0.91      0.90       242

   micro avg       0.90      0.90      0.90       500
   macro avg       0.90      0.90      0.90       500
weighted avg       0.90      0.90      0.90       500
 samples avg       0.90      0.90      0.90       500



*Finished*

Inception

In [67]:
baseInceptionModel = tf.keras.applications.InceptionV3(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseInceptionModel.trainable = False

In [68]:
avgPoolingLayerIn =  tf.keras.layers.GlobalAveragePooling2D()(baseInceptionModel.output)
predLayerIn = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerIn)

Inceptionmodel = tf.keras.models.Model(inputs = baseInceptionModel.input, outputs = predLayerIn)

In [69]:
Inceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Inceptionmodel.fit(xtrain, ytrain, epochs=2, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
13/13 [==============================] - 96s 7s/step - loss: 0.9622 - accuracy: 0.3078 - val_loss: 0.7661 - val_accuracy: 0.4880
Epoch 2/2
13/13 [==============================] - 87s 7s/step - loss: 0.6474 - accuracy: 0.6148 - val_loss: 0.5391 - val_accuracy: 0.7820


In [70]:
predIn = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Inceptionmodel.predict(rimg))
   if result == 0:
      predIn.append([1,0])
   if result == 1:
      predIn.append([0,1])

print(classification_report(predIn, ytest))

1/1 [==============================] - 0s 178ms/step
              precision    recall  f1-score   support

           0       0.70      0.84      0.76       208
           1       0.87      0.74      0.80       292

   micro avg       0.78      0.78      0.78       500
   macro avg       0.78      0.79      0.78       500
weighted avg       0.80      0.78      0.78       500
 samples avg       0.78      0.78      0.78       500



*Finished*

Xception

In [71]:
baseXceptionmodel = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
)

baseXceptionmodel.trainable = False

In [72]:
avgPoolingLayer3 =  tf.keras.layers.GlobalAveragePooling2D()(baseXceptionmodel.output)
predLayer3 = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer3)

Xceptionmodel = tf.keras.models.Model(inputs = baseXceptionmodel.input, outputs = predLayer3)

In [73]:
Xceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Xceptionmodel.fit(xtrain, ytrain, epochs=2, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
13/13 [==============================] - 166s 13s/step - loss: 0.6881 - accuracy: 0.5893 - val_loss: 0.6058 - val_accuracy: 0.7380
Epoch 2/2
13/13 [==============================] - 160s 13s/step - loss: 0.5802 - accuracy: 0.7775 - val_loss: 0.5284 - val_accuracy: 0.8660


In [74]:
pred3 = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Xceptionmodel.predict(rimg))
   if result == 0:
      pred3.append([1,0])
   if result == 1:
      pred3.append([0,1])
   

#print(pred3)
#print(ytest)

print(classification_report(pred3, ytest))

1/1 [==============================] - 0s 201ms/step
              precision    recall  f1-score   support

           0       0.86      0.87      0.87       248
           1       0.87      0.86      0.87       252

   micro avg       0.87      0.87      0.87       500
   macro avg       0.87      0.87      0.87       500
weighted avg       0.87      0.87      0.87       500
 samples avg       0.87      0.87      0.87       500



*Finished*

Summary all

In [75]:
print("MobileNetV2 report")
print(classification_report(pred, ytest))
print("########################################")

print("DenseNet121 report")
print(classification_report(predDen, ytest))
print("########################################")

print("Incception report")
print(classification_report(predIn, ytest))
print("########################################")

print("Xception report")
print(classification_report(pred3, ytest))
print("########################################")

MobileNetV2 report
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       253
           1       0.93      0.94      0.94       247

   micro avg       0.94      0.94      0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500
 samples avg       0.94      0.94      0.94       500

########################################
DenseNet121 report
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       258
           1       0.88      0.91      0.90       242

   micro avg       0.90      0.90      0.90       500
   macro avg       0.90      0.90      0.90       500
weighted avg       0.90      0.90      0.90       500
 samples avg       0.90      0.90      0.90       500

########################################
Incception report
              precision    recall  f1-score   support

           0       0.70      0.84      0.76  